# Existing Transit on the SHN

## Criteria
* Transit routes that run at least 20% of their route on the State Highway Network (SHN).
* slower than **25 mph**) **OR** 
* have **at least 5 trips per mile**

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import branca
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

from calitp_data_analysis import geography_utils 
from calitp_data_analysis import calitp_color_palette as cp
from bus_service_utils import better_bus_utils

catalog = intake.open_catalog("../bus_service_increase/*.yml")

# Modify rt_utils.ZERO_THIRTY_COLORSCALE to go up to 65 mph
ZERO_SIXTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=65)
ZERO_SIXTY_COLORSCALE.caption = "Speed (miles per hour)"

# Map args
TILES = "CartoDB positron"

CUSTOM_CATEGORICAL = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
    cp.CALITP_CATEGORY_BRIGHT_COLORS[5], # purple  
    cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
]

In [ ]:
speed = 25
trips = 5
gdf = better_bus_utils.select_highway_corridors(
    speed_dict = {"mean_speed_mph_trip_weighted": speed},
    trip_dict = {"trips_all_day_per_mi": trips}
)

In [ ]:
plot_df = gdf[
        gdf.route_length >= geography_utils.FEET_PER_MI * 0.5
    ].assign(
        geometry = (gdf.geometry.to_crs(geography_utils.CA_StatePlane)
                .buffer(300)
                .to_crs(geography_utils.WGS84)
               )
    ).reset_index(drop=True)

districts = sorted(gdf.District.unique().tolist())

In [ ]:
display(
    Markdown(
        f"There are **{len(gdf)}** five-mile corridors in the state. "
        f"<br>Of these, **{len(plot_df)}** corridors meet the criteria of being "
        f"less than {speed} mph **or** having at least {trips} daily trips per mile. "
    )
)

In [ ]:
def plot_highway_corridor(gdf: gpd.GeoDataFrame, 
                          district: int):
    """
    Returns a folium.Map using geopandas.explore()
    """
    
    keep_cols = ['Route', 'County', 'District', 'RouteType',
                 'trips_peak', 'trips_all_day', 
                 'trips_all_day_per_mi', 
                 'mean_speed_mph_trip_weighted', 
                 'geometry'
    ]
    
    
    plot_df = (gdf[(gdf.District == district)]
               [keep_cols].reindex(columns = keep_cols)
              )
        
    if len(plot_df) > 0:
        display(HTML(f"<h4> Average Speed </h4>"))

        m1 = plot_df.explore(
            "mean_speed_mph_trip_weighted", 
            # switch out colormap to allow higher speeds
            cmap = ZERO_SIXTY_COLORSCALE, 
            categorical=False, tiles = TILES)

        display(m1)
     
        display(HTML(f"<h4> Daily Trips</h4>"))
        
        m2 = plot_df.rename(columns = {"trips_all_day_per_mi": "Daily Trips per Mile"}).explore(
            "Daily Trips per Mile", 
            cmap = "viridis_r", #branca.colormap.linear.GnBu_09.scale(
            categorical=False, tiles = TILES)
    
        display(m2)
    else:
        print("No highway corridors meet this criteria.")
        

## District Maps

In [ ]:
for i in districts:
    display(HTML(f"<h3> District {i}</h3>"))
    
    plot_highway_corridor(plot_df, i)